In [ ]:
# default_exp backtest

# Backtesting 

Backtesting Team Strength Models

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
import os
import dotenv
import datetime as dt

import wingback.teamstrength

In [ ]:
#export
import abc
import itertools

import numpy as np
import mezzala

In [ ]:
#export


class MetricABC:
    @abc.abstractmethod
    def evaluate_one(self, test, predictions):
        """ Evaluate a single match """
        return 1.0
    
    def evaluate(self, test, predictions):
        """ Evaluate a set of matches """
        return [self.evaluate_one(t, p) for t, p in zip(test, predictions)]


class NLLScoreline(MetricABC):
    def evaluate_one(self, test, predictions):
        home_goals, away_goals = test['home_goals'], test['away_goals']
        scoreline_pred, *__ = [
            p for p in predictions
            if p.home_goals == home_goals
            and p.away_goals == away_goals
        ]
        return -np.log(scoreline_pred.probability)
    

class NLLOutcome(MetricABC):
    def evaluate_one(self, test, predictions):
        outcome = mezzala.scoreline_to_outcome(test['home_goals'], test['away_goals'])
        outcome_pred = mezzala.scorelines_to_outcomes(predictions)[outcome]
        return -np.log(outcome_pred.probability)

In [ ]:
#export


class Backtest:
    def __init__(self, models, metrics):
        self.models = models
        self.metrics = metrics
        
    def backtest(self, league_ids, dates):
        results = []
        for model, date in itertools.product(self.models, dates):
            train, test = model.fetch_data(league_ids, date)
            model.fit(train)
            predictions = model.predict(test)
            
            results.append({
                'model': str(model),  # idk...
                'date': date,
                'predictions': predictions,
                **{metric.__class__.__name__: metric.evaluate(test, predictions) 
                   for metric in self.metrics}
            })
        return results

In [ ]:
dotenv.load_dotenv()

wingback.db.queries.connect('postgresql://{user}:{password}@{host}:{port}/{database}'.format(
    host=os.environ['DB_HOST'],
    user=os.environ['DB_USER'],
    password=os.environ['DB_PASS'],
    database=os.environ['DB_NAME'],
    port=os.environ['DB_PORT'],
))

In [ ]:
backtest = Backtest(
    models=[
        wingback.teamstrength.DCGoals(),
    ],
    metrics=[
        NLLOutcome(),
        NLLScoreline()
    ]
)

results = backtest.backtest(
    league_ids=[1],
    dates=[dt.datetime(2020, 1, 1).date()]
)

/Users/ben/code/wingback/venv/lib/python3.7/site-packages/mezzala/models.py:124: RuntimeWarning: overflow encountered in multiply
  tau = np.where((home_goals == 0) & (away_goals == 0), 1 - home_rate*away_rate*rho, tau)
/Users/ben/code/wingback/venv/lib/python3.7/site-packages/mezzala/models.py:135: RuntimeWarning: invalid value encountered in log
  np.log(self._tau(home_goals, away_goals, home_rate, away_rate, rho))
